# Работа с Yandex Query в Yandex DataSphere

Работа с Yandex Query производится с помощью пакета [`yandex_query_magics`](https://github.com/yandex-cloud/yandex-query-magics), предоставляющего Jupyter notebook magic commands `%yq` и `%%yq`. 

1. [Подготовка окружения](#environment)
2. [Выполнение запросов](#basic_query)
3. [Захват переменных](#capture)
4. [Аргументы команд](#arguments)

Подробное описание работы с пакетом yandex_query_magics приведено в [документации Облака](https://cloud.yandex.ru/ru/docs/query/tutorials/jupyter#capture-command-result).

<a id='environment'></a>
## 1. Подготовка окружения

Для начала работы с Yandex Query необходимо установить пакет `yandex_query_magic`, который предоставляет magic command для работы в Jupyter.

In [ ]:
%pip install yandex_query_magic

Загрузите extension:

In [ ]:
%load_ext yandex_query_magic

Настройте параметры подключения к Yandex Query

- [Создайте](https://cloud.yandex.ru/ru/docs/iam/operations/authorized-key/create) авторизованный ключ доступа в консоли Облака с правами `editor` и [поместите его в переменную окружения Data Sphere](https://cloud.yandex.com/ru/docs/datasphere/operations/data/secrets) `yq_access_key`, как показано на изображении.

<img src="https://storage.yandexcloud.net/yq-public-resources/ds-jupyter-doc/new_secret.png" width="500"/>

- Установите фолдер, в котором вы хотите запускать запросы к Yandex Query, с помощью параметра `folder-id`

- Выполните команду настройки подключения

In [ ]:
%yq_settings --folder-id <идентификатор_фолдера> --env-auth yq_access_key

Создайте подключение к Object Storage. Подключение описывает информацию для авторизации в Object Storage. Для создания подключения укажите название существующего бакета в Object Storage. 

Данные примеров Yandex Query расположены в бакете, не требующем аутентификации, поэтому в пункте "Аутентификация бакета" выберите `Публичный`. Если же вы подключаетесь к собственному бакету с включенной авторизацией, выберите пункт "Приватный" и выберите в пункте "Сервисный аккаунт" аккаунт, от имени которого будет выполняться доступ к данным внутри бакета. 

<img src="https://storage.yandexcloud.net/yq-public-resources/ds-jupyter-doc/create-tutorial-connection.png" width="500"/>

Создайте привязку к данным в Object Storage. Привязка к данным содержит информацию о форматах и расположении файлов в бакете, а также о списке полей в данных и их типах.

Для создания привязки к данным указываются название подключения, которое будет использоваться для доступа к данным, путь к данным и формат данных внутри бакета Object Storage, а также список колонок внутри файлов.

Настройте привязку к данным, как показано на следующем изображении. После ввода всех параметров нажмите кнопку "Предпросмотр" для проверки, что все параметры указаны корректно. 

<img src="https://storage.yandexcloud.net/yq-public-resources/ds-jupyter-doc/create-tutorial-binding.png" width="500"/>

Выполните тестовый запрос:

In [ ]:
%yq SELECT * FROM yq_tutorial LIMIT 100;

<a id='basic_query'></a>
## 2. Выполнение запросов

Запросы к Yandex Query можно выполнять с помощь line-команды `%yq` или с помощью cell-команды `%%yq`. С помощью line-команды `%yq` удобно выполнять однострочные запросы, а с помощью cell-команды `%%yq` запросы любого размера.

In [ ]:
%yq SELECT * FROM yq_tutorial LIMIT 100;

In [ ]:
%%yq
SELECT 
    * 
FROM 
    `yq_tutorial` LIMIT 100;

Результат запроса можно присвоить переменной Python с помощью операции присваивания. По умолчанию результатом исполнения команды является объекты типа `pandas.DataFrame`.

In [ ]:
result = %yq SELECT * FROM yq_tutorial LIMIT 100;

In [ ]:
print(result.info())

Присваивать результат исполнения в переменную можно и для cell-команды. 

In [ ]:
result = %%yq 
SELECT 
  * 
FROM `yq_tutorial` LIMIT 100;

<a id='capture'></a>

## 3. Захват переменных

При работе с Yandex Query можно выполнять "захват переменных" при указании текста запроса. Захват переменных это режим работы, когда объявленная вне запроса переменная Jupyter используется как часть запроса. В примере ниже отдельно объявляется переменная `table_name`, а потом она используется как часть текста запроса.

In [ ]:
table_name = "yq_tutorial"

In [ ]:
%yq SELECT * FROM {{table_name}} LIMIT 100

Захватывать можно не только простые типы данных, но и сложные: списки, словари. В этом случае python-типы данных будут переведены в соответствующие конструкции языка [YQL](https://ydb.tech/docs/ru/yql/reference/). Например, Python list будет переведен в YQL-тип `List`, а SQL запрос ниже будет переведен в следующее YQL-выражение: `SELECT * FROM yq_tutorial where trip_distance in AsList(7.6,2.22) LIMIT 100`.

In [ ]:
distances = [7.60, 2.22]

In [ ]:
%yq SELECT * FROM yq_tutorial where trip_distance in {{distances}} LIMIT 100

Кроме списков можно захватывать и словари Python.

In [ ]:
dct = {"a": "1", "b": "2", "c": "test", "d": "4"}

In [ ]:
%yq SELECT "a" in {{dct}}

<a id='arguments'></a>

## 4. Аргументы команд

При работе с cell и line-командами можно использовать дополнительные аргументы.
- --name "Название запроса"
- --raw-results


In [ ]:
%yq --name "Название запроса" --raw-results SELECT "Hello, world!" 

Подробнее про работу с Yandex Query из Jupyter Notebook'ов описано в [документации Yandex Query](https://cloud.yandex.ru/ru/docs/query/tutorials/jupyter).